In [24]:
import requests as req 
import json

In [25]:
url = 'http://localhost:11434/api/chat'

def get_completion(url, messages):
    result = ""
    s = req.Session()
    data = {
        "model": "tinyllama",
        "messages": messages,
        "stream" : False
    }
    with s.post(url, data=json.dumps(data), headers=None, stream=False) as resp:
        if resp.status_code != 200:
            raise Exception(f"Failed to get completion: {resp.status_code}")
        else: 
            return resp.json() 


### Sensible Column Match

Using the LLM to test whether the example column is a sensible match with the given existing column

In [26]:
class ColumnProfile:
    def __init__(self, col_id, table_name, name, data_type, total_values, unique_values, non_empty_values):
        self.id = col_id
        self.table_name = table_name
        self.name = name
        self.data_type = data_type
        self.total_values = total_values
        self.unique_values = unique_values
        self.non_empty_values = non_empty_values

    def from_json(json):
        return ColumnProfile(json['id'], json['sourceName'], json['columnName'], json['dataType'], json['totalValues'], json['uniqueValues'], json['nonEmptyValues'])
    
class ExampleColumn:
    def __init__(self, attr: str, examples: list[str]) -> None:
        self.attr = attr
        self.examples = examples

In [27]:
def score_candidate(example_column, column_profile):
    messages = [
        {  
            "role" : "system",
            "content" : """You are an assistant whose job is to help a database user find a view that contains relevant information. 
                    The user has a hypothetical column which illustrates a column they would like their view to contain.
                    The user has a column they are considering. Provide a score from 1 to 100 for how likely the column is to be the
                    column that the user is looking for. Do not provide other reasoning."""
        },
        {
            "role" : "user", 
            "content" : "Example, hypothetical column " + example_column.attr + "."
        },
        {   
            "role" : "user",
            "content" : "The actual database column is in the table " + column_profile.table_name + " and is called " + column_profile.name + "."
        }, 
        {
            "role" : "user",
            "content" : "The database column contains " + str(column_profile.total_values) + " values, " + str(column_profile.unique_values) + " of which are unique, and " + str(column_profile.non_empty_values) + " of which are non-empty."
        },
        {
            "role" : "user",
            "content" : "The database column is of type " + column_profile.data_type + "."
        },
        {
            "role" : "system",
            "content" : "Give this column a score from 1 to 100 "
        }
    ]

    completion = get_completion(url, messages)
    return completion['message']['content']

In [28]:
with open('column_profile.json') as f:
    candidate = ColumnProfile.from_json(json.load(f))

example = ExampleColumn("address", ["772 Fort Worth Ave", "389 E 48th St"])

print(score_candidate(example, candidate))

Here's an example for the Hypothetical Column:

For Hypothetical Column, address column contains 647 unique values and 647 non-empty values. Score is given by the following formula:

Score = (Number of Unique Values / Number of Non-Empty Values) x 100


In [35]:
def score_candidate_reduced(example_column, column_profile):
    messages = [
        {  
            "role" : "user",
            "content" : f"""I am looking at a database for a column that contains information of type {example_column.attr}."""
        },
        {   
            "role" : "user",
            "content" : f"The table \"{column_profile.table_name}\" has a column \"{column_profile.name}\"" # of type {column_profile.data_type} with  {column_profile.total_values} values, {column_profile.unique_values} of which are unique, and {column_profile.non_empty_values} of which are non-empty."
        },
        {
            "role" : "user",
            "content" : "Is this probably the correct column? Respond only with \'likely\', \'unlikely\', or \'unsure\'."
        }
    ]

    completion = get_completion(url, messages)
    return completion['message']['content']

In [36]:
with open('column_profile.json') as f:
    candidate = ColumnProfile.from_json(json.load(f))

example = ExampleColumn("address", ["772 Fort Worth Ave", "389 E 48th St"])

print(score_candidate_reduced(example, candidate))

Based on the information provided, I'd say that "address" is a likely column for the table "Chicago Public Schools - School Location Data SY2122.csv". However, as this is not an official database of Chicago Public Schools, it is essential to double-check the specific details before making any data analysis or predictions based on these records. It's always a good practice to verify information in real-time and update your findings as new information becomes available.


In [37]:
with open('column_profile.json') as f:
    candidate = ColumnProfile.from_json(json.load(f))

example = ExampleColumn("address", ["772 Fort Worth Ave", "389 E 48th St"])

print(score_candidate_reduced(example, candidate))

Based on the information provided, it is most likely that the "address" column in the Chicago Public Schools - School Location table from SY2122.csv represents a location address. However, as the text provides no contextual information about this data or any other tables within the same database, it's not entirely clear whether this is the correct column for this data. It's best to consult the relevant documentation or ask the original source for further clarification.


### Chat GPT 

In [41]:
from openai import OpenAI
client = OpenAI()

def get_gpt_completion(messages, max_tokens=1000):
    
    response = client.chat.completions.create(
        model='gpt-4-0125-preview',
        messages=messages,
        max_tokens=max_tokens)
    
    return response.choices[0].message.content

In [39]:
def score_candidate_gpt(example_column, column_profile):
    messages = [
        {  
            "role" : "user",
            "content" : f"""I am looking at a database for a column that contains information of type {example_column.attr}."""
        },
        {   
            "role" : "user",
            "content" : f"The table \"{column_profile.table_name}\" has a column \"{column_profile.name}\" of type {column_profile.data_type} with  {column_profile.total_values} values, {column_profile.unique_values} of which are unique, and {column_profile.non_empty_values} of which are non-empty."
        },
        {
            "role" : "user",
            "content" : "Could this be the correct column?"
        }
    ]
    return get_gpt_completion(messages)


In [42]:
with open('column_profile.json') as f:
    candidate = ColumnProfile.from_json(json.load(f))

example = ExampleColumn("address", ["772 Fort Worth Ave", "389 E 48th St"])

print(score_candidate_gpt(example, candidate))

Based on the description you provided for the "Chicago Public Schools - School Locations SY2122.csv" dataset:

- **Column Name**: "address"
- **Data Type**: T (Typically, "T" denotes a text/string data type)
- **Total Values**: 647
- **Unique Values**: 625
- **Non-Empty Values**: 647

Given these characteristics, it strongly indicates that this "address" column is indeed likely to contain address information for each school listed in the dataset. Addresses are typically unique to each location, which is consistent with the high number of unique values (625 out of 647). Also, the fact that it is of text type ('T') and all entries are non-empty supports the notion that this column accurately represents address information.

So yes, this appears to be the correct column for obtaining address information of Chicago Public Schools from the mentioned dataset for the school year 2021-2022.


### Incorrect Join Path

In [43]:
placement_column = ColumnProfile(2341, "Chicago Public Schools - School Performance Reports", "Placement", "text", 647, 12, 32)
enrollment_column = ColumnProfile(23481, "Chicago Public Schools - School Enrollment", "Enrollment", "integer", 647, 622, 3)

In [45]:
# 'placement' can be conflated with location 
print(score_candidate_reduced(example, placement_column))

I do not have access to specific information about the chicago public schools - school performance reports table. However, based on common practices in the academic and research community, "placement" is a commonly used term for the designation of the attendance area or district within which a particular school operates. In most cases, "placement" refers to the geographic location of the school, rather than its grade level or other administrative designation. Therefore, it's possible that this column could contain information about both attendance areas and school districts, but based on the specific context provided, we can safely assume it's likely that it's referring to a combination of both.


In [46]:
# enrollment is nothing like address 
print(score_candidate_reduced(example, enrollment_column))

I do not have access to specific data and cannot make a judgment on whether this column is likely, unlikely, or unsure. However, given that it's from "chicago public schools - school enrollment," we can assume that it contains information about the number of students enrolled in a particular school within a district/school district. This would indicate an 'unlikely' or 'unsure' answer based on the context.
